# Filtering Spam with Naive Bayes

#### Overview

The goal of this project is to test the effectiveness of the Multinomial Naive Bayes classification algorithm as an spam filter for SMS messages. This technique is not limited to SMS messages and could be applied to a broad range of text classification problems (emails, comments, product reviews, etc.) given that training data with labels is available. While this case aims to predict one of two labels (spam/non-spam), the model could be expanded to cases with a greater number of labels as well.

The dataset that I am using for testing comes from Tiago A. Almeida and José María Gómez Hidalgo and is hosted on [the UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). It includes 5,572 SMS messages which have been classified by humans as either "spam" or "ham" (non-spam).

#### Results

The naive bayes model was a highly effective predictor for this dataset, achieving 98.8% label accuracy on the test partition of the dataset.

||Actual Spam|Actual Ham|Total|
|--|--|--|--|
|Predicted Spam|139|5|<b>  144|
|Predicted Ham|8|963|<b>  971|
|<b>Total|<b>147|<b>968|<b> 1155|


#### Discussion

One of the major benefits of this model was how easy it was to set it up. It required only a small amount of text cleaning and manupulation with Pandas to get the data in a usable format. The classifier itself is just a custom is just a basic conditional probability calculation and requires no additional libraries.

On the other hand, the simplicity of the model makes it somewhat difficult to tweak and customize for different circumstances. For example, if there is a situation in which false positives are unacceptable, there isn't an obvious  parameter to adjust to achieve this. A workaround could be to adjust the comparison function so that the P(spam | message) must be some % higher than P(non-spam | message), but this would require some trial and error to find a suitable amount.

A potential issue with this model is that as the word count of a given message increases by n, the number of probabilities being multiplied together in the comparison formulas also increases by n. This causes the resulting number to grow smaller and given a high enough word count could lead to underflow and innaccurate predictions. One potential work around for this is to convert the calculations to a log scale.

## Libraries

In [1]:
import pandas as pd
import re
pd.set_option('display.max_colwidth', 65)

## Data

#### Import

In [2]:
df = pd.read_csv("SMSSpamCollection", sep="\t", header=None, names=['Label', 'SMS'])

In [3]:
print(df.head(10))

  Label                                                               SMS
0   ham  Go until jurong point, crazy.. Available only in bugis n grea...
1   ham                                     Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup final tkts 21st May...
3   ham                 U dun say so early hor... U c already then say...
4   ham     Nah I don't think he goes to usf, he lives around here though
5  spam  FreeMsg Hey there darling it's been 3 week's now and no word ...
6   ham  Even my brother is not like to speak with me. They treat me l...
7   ham  As per your request 'Melle Melle (Oru Minnaminunginte Nurungu...
8  spam  WINNER!! As a valued network customer you have been selected ...
9  spam  Had your mobile 11 months or more? U R entitled to Update to ...


#### Summary Stats

 - 5,572 total messages.
 - 13.4% of the messages are classified as spam.
 - Avg. length of SMS: 80 characters
 - Longest SMS: 910 characters
 - Shortest SMS: 2 characters
 - 403 messages are duplicates (see duplicates section)

In [4]:
print(df.shape)
print(df["Label"].value_counts(normalize=True))
print(df.SMS.map(lambda x: len(x)).mean())
print(df.SMS.map(lambda x: len(x)).max())
print(df.SMS.map(lambda x: len(x)).min())
print(df.duplicated().sum())

(5572, 2)
ham     0.865937
spam    0.134063
Name: Label, dtype: float64
80.48994974874371
910
2
403


#### Duplicate Values

This dataset contains 403 duplicate SMS messages. In this case, it doesn't necessarily mean that they are the same message recorded twice, but rather they could be the same message being sent at different times or between different people. It makes sense to see duplicates in spam because they are automated and sent to multiple people. In the case of "ham" messages, these might be generic replies such as "Sorry, I'll call later." or information sent to multiple people.

I've decided to retain the records since they came from a natural sampling processes and not an overlap in the data. This may introduce some slight bias by overrepresenting the words in these messages. In this case, the duplicate messages are 23% spam whereas spam messages make up only 13% of the dataset. 

Another way to look at this is that since the real world contains duplicate messages, it is probably appropriate that the training set keep the duplicate messages as well. [Interesting discussion on the topic.](https://stackoverflow.com/questions/26197700/should-i-keep-remove-identical-training-examples-that-represent-different-object)


In [5]:
duplicates = df[df.duplicated() == True]
print(duplicates["Label"].value_counts(normalize=True))
print(duplicates[duplicates["Label"] == 'ham'].head(10))
print("\n")
print(duplicates[duplicates["Label"] == 'spam'].head(10))

ham     0.766749
spam    0.233251
Name: Label, dtype: float64
    Label                                                               SMS
103   ham  As per your request 'Melle Melle (Oru Minnaminunginte Nurungu...
154   ham  As per your request 'Melle Melle (Oru Minnaminunginte Nurungu...
207   ham  As I entered my cabin my PA said, '' Happy B'day Boss !!''. I...
223   ham                                            Sorry, I'll call later
326   ham                                  No calls..messages..missed calls
339   ham                                            Sorry, I'll call later
444   ham                                            Sorry, I'll call later
533   ham                                 Gudnite....tc...practice going on
655   ham                                      Did u got that persons story
658   ham                              You will be in the place of that man


     Label                                                               SMS
357   spam  Congratulat

## Training / Test Split

The data will randomized and 20% of messages withheld for testing without cross-validation. The resulting samples have a similar label distribution to the full dataset.

In [6]:
# Randomize
df = df.sample(frac=1, random_state=1)

In [7]:
# Split
train = df.iloc[:4457].copy().reset_index(drop=True)
test = df.iloc[4457:].copy().reset_index(drop=True)

In [8]:
print(train["Label"].value_counts(normalize=True))
print(test["Label"].value_counts(normalize=True))

ham     0.86538
spam    0.13462
Name: Label, dtype: float64
ham     0.868161
spam    0.131839
Name: Label, dtype: float64


## Data Cleaning and Formating

In this implementation, I will be standardizing the SMS texts to lowercase and removing punctuation. A more nuanced model, if needed, could perhaps give consideration to these factors.

I will also be spliting each word in the vocabulary (all words used in training) to columns. The value of each column will be the number of times in the SMS text that the individual word was used. This will make it easy to find and plug in the word counts needed for scoring. Example:

|Label|'Secret'|'Prize'|'Claim'|'Money'|'Now'|
|---|---|---|---|---|---|
|Spam|0|1|1|1|1|
|Ham|0|0|0|0|1|
|Ham|1|0|0|0|0|

In [9]:
## Data clean
train['SMS'] = train['SMS'].str.replace('\W', ' ')
train['SMS'] = train['SMS'].str.lower().str.split()

vocabulary = []
for row in train['SMS']:
    for word in row:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))

In [10]:
train["SMS"].head(10)

0                                   [yep, by, the, pretty, sculpture]
1                [yes, princess, are, you, going, to, make, me, moan]
2                                     [welp, apparently, he, retired]
3                                                            [havent]
4    [i, forgot, 2, ask, ü, all, smth, there, s, a, card, on, da, ...
5    [ok, i, thk, i, got, it, then, u, wan, me, 2, come, now, or, ...
6    [i, want, kfc, its, tuesday, only, buy, 2, meals, only, 2, no...
7                                     [no, dear, i, was, sleeping, p]
8                                          [ok, pa, nothing, problem]
9                                    [ill, be, there, on, lt, gt, ok]
Name: SMS, dtype: object

In [11]:
word_counts_per_sms = {unique_word: [0] * len(train['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [12]:
train_counts = pd.DataFrame.from_dict(word_counts_per_sms)

In [13]:
training_df = pd.concat([train, train_counts], axis=1)

## Calculate Constants

Multinomial naive bayes uses the following calculations to score each incoming message based on words previously observed in the training data. The formula with the higher resulting score determines whether the message will be classified as either Spam or Ham.

$$
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
$$$$
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
$$

 $$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$


In [14]:
## Split dataframes by label
ham_df = training_df[training_df["Label"] == "ham"].copy()
spam_df = training_df[training_df["Label"] == "spam"].copy()

In [15]:
## Probability that message is ham or spam
p_ham = ham_df.shape[0] / training_df.shape[0]
p_spam = spam_df.shape[0] / training_df.shape[0]

## Number of total words
n_vocab_total = len(vocabulary)

## Number of words in spam messages
n_vocab_given_spam = spam_df['SMS'].apply(len)
n_vocab_given_spam = n_vocab_given_spam.sum()

# Number of woords in ham messages
n_vocab_given_ham = ham_df["SMS"].apply(len)
n_vocab_given_ham = n_vocab_given_ham.sum()

## Laplace Smoothing
alpha = 1


In [16]:
print(p_spam)
print(p_ham)
print(n_vocab_total)
print(n_vocab_given_spam)
print(n_vocab_given_ham)


0.13461969934933812
0.8653803006506618
7782
15190
57233


## Calculate Parameters

In addition to constants, naive bayes relies on a few parameters that can be pre-calculated to save time in classification. 

This section creates two dictionaries, one for messages labeled ham and one for spam. Each contains each word from the training vocabulary and the probability that the word is found in a message given that it is spam or ham.

The "calculate parameters" cell calculates these functions and loads them into the dictionaries:

 $$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$

In [17]:
## Initialize parameter hash tables
parameters_ham = {unique_word:0 for unique_word in vocabulary}
parameters_spam = {unique_word:0 for unique_word in vocabulary}

In [18]:
## Calculate parameters
for word in vocabulary:
    ## P(w_i|Ham)
    n_word_given_ham = ham_df[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_vocab_given_ham + alpha * n_vocab_total)
    parameters_ham[word] = p_word_given_ham
    
    ## P(w_i|Spam)
    n_word_given_spam = spam_df[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_vocab_given_spam + alpha * n_vocab_total)
    parameters_spam[word] = p_word_given_spam


## Classification function

With constants and parameters calculated, messaages can now be classified. In the classify function, each of the comparison formulas starts out as the observed percentages of spam/ham messages in the training data. For each word, the (P_w | Ham) and (P_w | Spam) are calculated and multiplied to the initial percentage. If the word is not found in the training vocabulary, then it is ignored. The formula with the largest total at the end of the message determines the classification.

In testing a few new messages, the classifier looks promising.

|Label|Message|
|---|---|
|Spam|CONGRATULATIONS! You have been selected for a secret prize.|
|Ham|Thanks for your message.|
|Ham|We need more Bort license plates in the gift shop.|


In [19]:
def classify(message):
    
    # Text clean up
    message = re.sub("\W", " ", message).lower().split()
    
    # Initial guess -> prior probabilities from training set.
    p_ham_given_message = p_ham
    p_spam_given_message = p_spam
    
    # Adjust probabilities for each word in new message:
    for word in message:
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
    if p_spam_given_message > p_ham_given_message:
        return "spam"
    else:
        return "ham"


In [20]:
## Gut check
test_spam = 'CONGRATULATIONS. You have been selected for a secret prize.'
test_ham = "Thanks for your message."
test_simpsons_quote = 'We need more Bort license plates in the gift shop.'

print(classify(test_spam))
print(classify(test_ham))
print(classify(test_simpsons_quote))

spam
ham
ham


## Predictions and Accuracy

In [21]:
# Predict
test["prediction"] = test["SMS"].apply(classify)

In [22]:
# Percentage accuracy
correct_pct = round(((test["prediction"] == test["Label"]).sum() / test.shape[0]) * 100, 2)
print(str(correct_pct) + "%")

98.83%


#### Decomposing accuracy

In [23]:
# False Positives - Predicted a ham message to be spam.
false_positives = test[(test["prediction"] == "spam") & (test["Label"] == "ham")]
print(str(false_positives.shape[0]) + " false positives")
false_positives

5 false positives


,Label,SMS,prediction
153,ham,Unlimited texts. Limited minutes.,spam
160,ham,26th OF JULY,spam
285,ham,Nokia phone is lovly..,spam
303,ham,No calls..messages..missed calls,spam
320,ham,We have sent JD for Customer Service cum Accounts Executive t...,spam


In [24]:
# False Negatives - Predicted a spam message to be ham.
false_negatives = test[(test["prediction"] == "ham") & (test["Label"] == "spam")]
print(str(false_negatives.shape[0]) + " false negatives")
false_negatives

8 false negatives


,Label,SMS,prediction
115,spam,Not heard from U4 a while. Call me now am here all night with...,ham
136,spam,More people are dogging in your area now. Call 09090204448 an...,ham
505,spam,"Oh my god! I've found your number again! I'm so glad, text me...",ham
547,spam,"Hi babe its Chloe, how r u? I was smashed on saturday night, ...",ham
742,spam,"0A$NETWORKS allow companies to bill for SMS, so they are resp...",ham
877,spam,RCT' THNQ Adrian for U text. Rgds Vatian,ham
886,spam,2/2 146tf150p,ham
954,spam,Hello. We need some posh birds and chaps to user trial prods ...,ham


In [31]:
# True Positives - Predicted spam for spam message
true_positives = test[(test["Label"] == "spam") & (test["prediction"] == "spam")]
print(str(true_positives.shape[0]) + " true positives")

139 true positives


In [32]:
# True Negatives - Predicted ham for ham message
true_negatives = test[(test["Label"] == "ham") & (test["prediction"] == "ham")]
print(str(true_negatives.shape[0]) + " true negatives")

963 true negatives
